In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(f'TensorFlow Version : {tf.__version__}')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_img_path = '../input/landmark-recognition-2021/train/'
test_img_path = '../input/landmark-recognition-2021/test/'

In [ ]:
traincsv = pd.read_csv('../input/landmark-recognition-2021/train.csv')
traincsv['path'] = traincsv['id'].transform(lambda x : train_img_path+x[0]+'/'+x[1]+'/'+x[2]+'/'+x+'.jpg')
traincsv.head(1)

In [ ]:
top_freq = traincsv['landmark_id'].value_counts().head(5).index
df = traincsv.loc[traincsv['landmark_id'].isin(top_freq)].copy()
df.head(1)

In [ ]:
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['landmark_id'])
df.head()

In [ ]:
dftrain, dfval = train_test_split(df, test_size=0.2)

In [ ]:
def decode_img(dataset, trainable):
    if trainable:
        return dataset.map(lambda x, y : (tf.image.resize_with_pad(
                    tf.io.decode_image(tf.io.read_file(x), channels=3), 
                                    150, 150)/255., y),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
    else:
        return dataset.map(lambda x : tf.image.resize_with_pad(
                    tf.io.decode_image(tf.io.read_file(x), channels=3), 
                                    150, 150)/255.,
        num_parallel_calls=tf.data.experimental.AUTOTUNE
        )

In [ ]:
def get_ds(df, shuffle=True, trainable=True):
    if trainable:
        labels = df['label']
        ds = tf.data.Dataset.from_tensor_slices((df['path'], labels))
    else:
        ds = tf.data.Dataset.from_tensor_slices(df['path'])
    ds = decode_img(ds, trainable)
    if shuffle:
        ds = ds.shuffle(len(df))
    ds = ds.batch(64)
    ds = ds.prefetch(2)    
    return ds

dstrain = get_ds(dftrain)
dsval = get_ds(dfval, shuffle=False)

In [ ]:
inputs = tf.keras.Input(shape=(150,150,3))
x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['sparse_categorical_accuracy'])

model.summary()

In [ ]:
model.fit(dstrain, epochs=5)

In [ ]:
model.evaluate(dsval)

In [ ]:
forecast = model.predict(dsval)
print(f'First Data Label : {np.argmax(forecast[0])}')
dfval['label'].head(1)